<h1><center> Text mining part </center></h1>

In [1]:
import requests
from bs4 import BeautifulSoup

In [75]:
def remove_spaces(text):
    return text.replace(" ", "")
def get_numbers_from_text(text):
    import re
    text = remove_spaces(text)
    pattern = '[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?'
    list_of_numbers = re.findall(pattern, text)
    return list_of_numbers
def find_years(numbers):
    numbers = [x for x in numbers if (float(x) > 1980) and (float(x) < 2030)]
    return numbers
def find_km(numbers):
    numbers = [x for x in numbers if (float(x) > 3000) and (float(x) < 500000)]
    return numbers
def get_context(text, list_of_tokens, year_dictionary = ['egistr', 'r.v', 'vyrob', 'výrob', 'prov'], km_dictionary = ['km', 'Km', 'KM', 'ilomet', 'ajet', 'ájez', 'achom', 'atoč'], context_span=20):
    import re
    context = []
    year = 'No match'
    km = 'No match'
    for token in find_years(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in year_dictionary:
                year_find = [m.start() for m in re.finditer(s, substring)]
                if len(year_find) > 0:
                    year = token
    for token in find_km(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in km_dictionary:
                km_find = [m.start() for m in re.finditer(s, substring)]
                if len(km_find) > 0:
                    km = token
    return [year, km]
def get_info(links):
    results = []
    for i in links:
        add_page = requests.get(i)
        soup_add = BeautifulSoup(add_page.text, 'html')
        add = remove_spaces(soup_add.find('div', {'class':'popisdetail'}).get_text())
        all_numbers = get_numbers_from_text(add)
        context_got = get_context(add, all_numbers)
        result = [i, context_got[0], context_got[1]]
        results.append(result)
    return results
link = ['https://auto.bazos.cz/inzerat/157012769/vw-golf-7-20tdi-110kw-107tkm-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156954743/skoda-octavia-3dsg-20-tdi.php']
get_info(link)

[['https://auto.bazos.cz/inzerat/157012769/vw-golf-7-20tdi-110kw-107tkm-plna-zaruka-2-roky-zdarma.php',
  '2018',
  '107604'],
 ['https://auto.bazos.cz/inzerat/156954743/skoda-octavia-3dsg-20-tdi.php',
  '2013',
  '207000']]